In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


/home/danilo/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
colunas = ['cd_causa', 'cd_atendimento', 'ds_Acao_Judicial', 'ds_fatos', 'ds_Pedidos', 'ds_Qualificacao']

In [3]:
# LINHA DE SELECAO DO INPUT
df = pd.read_csv(
    "dataset_clinica20252.csv",
    sep="|",               
    encoding="utf-8"     
)

print(df.shape)
print(df.head(20))

(19800, 6)
      cd_causa             cd_atendimento                 ds_Acao_Judicial  \
0   CIB0500064  0825789-84.2025.8.18.0140  90 - ACAO DE REPARACAO DE DANOS   
1   CIB0505587  1004697-72.2025.8.26.0066  90 - ACAO DE REPARACAO DE DANOS   
2   CIB0508201  0800423-07.2025.8.15.0761  90 - ACAO DE REPARACAO DE DANOS   
3   CIB0514647  1004875-69.2025.8.26.0438  90 - ACAO DE REPARACAO DE DANOS   
4   CIB0500604  0010630-50.2025.8.27.2706  90 - ACAO DE REPARACAO DE DANOS   
5   CIB0511068  0092601-36.2025.8.05.0001  90 - ACAO DE REPARACAO DE DANOS   
6   CIB0515030  0801575-51.2025.8.18.0068  90 - ACAO DE REPARACAO DE DANOS   
7   CIB0515018  0801452-38.2025.8.18.0073  90 - ACAO DE REPARACAO DE DANOS   
8   CIB0515702  0800895-81.2025.8.10.0038  90 - ACAO DE REPARACAO DE DANOS   
9   CIB0514229  0148716-17.2025.8.04.1000  90 - ACAO DE REPARACAO DE DANOS   
10  CIB0514230  0148734-38.2025.8.04.1000  90 - ACAO DE REPARACAO DE DANOS   
11  CIB0524760  0161843-22.2025.8.04.1000  90 - ACAO 

In [42]:
df_validado_true = df.loc[
    df['ds_fatos'].str.contains("A parte Autora é beneficiária da Previdência Social, sendo tal sua única fonte de renda.  Portanto, por", 
    na=False)]
print(df_validado_true.shape)
print(df_validado_true.head())

(1, 6)
        cd_causa             cd_atendimento                 ds_Acao_Judicial  \
1744  CIB0557065  0001471-80.2025.8.04.6600  90 - ACAO DE REPARACAO DE DANOS   

                                               ds_fatos  \
1744  A parte Autora é beneficiária da Previdência S...   

                                             ds_Pedidos  \
1744  c1) ser declarado a inexistência dos débitos r...   

                                        ds_Qualificacao  
1744  EXCELENTÍSSIMO SENHOR DOUTOR JUIZ DE DIREITO D...  


In [46]:
from pinecone import Pinecone, ServerlessSpec
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain.schema import Document
import os
from dotenv import load_dotenv

load_dotenv(override=True)

pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY")) 

index_name = "clinicas"
if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=3072,
        metric='cosine',
        spec=ServerlessSpec(cloud='aws', region='us-east-1')
    )

index = pc.Index(index_name)

index.delete(delete_all=True)

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

docs = []
for text in df_validado_true['ds_fatos'].dropna():
    doc = Document(page_content=text, metadata={"label": "SIM"})
    docs.append(doc)

print(len(docs))
print(docs[0])

vector_store = PineconeVectorStore.from_documents(
    docs,
    embeddings,
    index_name=index_name
)


1
page_content='A parte Autora é beneficiária da Previdência Social, sendo tal sua única fonte de renda.  Portanto, por ser aposentada pelo INSS, a parte autora fica vulnerável a ter em sua folha  de pagamento, descontos indevidos, como aqueles oriundos de Empréstimos  Consignados, e todos os demais, que podem ser realizados em folha, mediante a simples  posse dos dados cadastrais do Requerente.  Ocorre que a parte Requerente, ao analisar sua folha de pagamento, pode perceber que,  vem lhe sendo descontado, valores referentes a empréstimo consignado que sequer tem  conhecimento e muito menos utilizou, conforme informações abaixo extraídas do portal  MEU INSS:' metadata={'label': 'SIM'}


In [47]:
def exemplos_contexto(query: str):
    resultados = vector_store.similarity_search_with_score(query, k=4)
    positivos, negativos = [], []
    for doc, score in resultados:
        exemplo = {"texto": doc.page_content, "score": score}
        if doc.metadata["label"] == "SIM":
            positivos.append(exemplo)
        else:
            negativos.append(exemplo)
    return positivos[:2], negativos[:2]

In [48]:
print(exemplos_contexto("A parte Autora é beneficiária da Previdência Social, sendo tal sua única fonte de renda.  Portanto, por"))

([{'texto': 'A parte Autora é beneficiária da Previdência Social, sendo tal sua única fonte de renda.  Portanto, por ser aposentada pelo INSS, a parte autora fica vulnerável a ter em sua folha  de pagamento, descontos indevidos, como aqueles oriundos de Empréstimos  Consignados, e todos os demais, que podem ser realizados em folha, mediante a simples  posse dos dados cadastrais do Requerente.  Ocorre que a parte Requerente, ao analisar sua folha de pagamento, pode perceber que,  vem lhe sendo descontado, valores referentes a empréstimo consignado que sequer tem  conhecimento e muito menos utilizou, conforme informações abaixo extraídas do portal  MEU INSS:', 'score': 0.670984268}], [])


In [34]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from enum import Enum
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

class BinaryChoice(str, Enum):
    SIM = "SIM"
    NAO = "NAO"

class Classificador(BaseModel):
    binary_score: BinaryChoice = Field(
        description="SIM para crédito consignado como causa raiz, NAO para documentos com outra causa principal"
    )


In [54]:
def classificar_linhas(texto: str, ctx_true, ctx_false) -> str:
    model = ChatOpenAI(model="gpt-4o", temperature=0)

    parser = PydanticOutputParser(pydantic_object=Classificador)
    
    prompt_template = '''
    Você é um especialista em análise de documentos jurídicos brasileiros.

    Analise o texto abaixo e classifique se o caso trata de CRÉDITO CONSIGNADO.

    CRÉDITO CONSIGNADO é caracterizado por:
    - Contratação VOLUNTÁRIA de empréstimo com desconto em folha de pagamento
    - Autor SOLICITOU o empréstimo e AUTORIZOU os descontos
    - Discussão sobre TAXAS DE JUROS, PRAZOS ou CONDIÇÕES do empréstimo consignado
    - Margem consignável excedida
    - Problemas com portabilidade de crédito consignado

    NÃO é crédito consignado:
    - Descontos NÃO AUTORIZADOS de seguros, cartões ou serviços bancários
    - Cobranças indevidas de produtos não contratados
    - Venda casada
    - Negativação indevida
    - Cartão de crédito comum (não consignado)

    Use como contexto os exemplos abaixo:
    Exemplos POSITIVOS (CRÉDITO CONSIGNADO):
    {ctx_true}

    Exemplos NEGATIVOS (NÃO CRÉDITO CONSIGNADO):
    {ctx_false}

    Utilize o score dos exemplos para ajudar na decisão.

    Texto para analisar: {texto}

    {format_instructions}
    '''
        
    prompt = ChatPromptTemplate.from_template(
        template=prompt_template,
        partial_variables={"format_instructions": parser.get_format_instructions()}
    )
    

    chain = prompt | model | parser
    
    
    resultado = chain.invoke({"texto": texto, "ctx_true": ctx_true, "ctx_false": ctx_false})
    
    return resultado.binary_score.value

In [ ]:
texto = df_validado_true['ds_fatos'].iloc[0]

pos, neg = exemplos_contexto(texto)
ctx_true = "\n".join([f"{ex['texto']} - score: {ex['score']}" for i, ex in enumerate(pos)])
ctx_false = "\n".join([f"{ex['texto']} - score: {ex['score']}" for i, ex in enumerate(neg)])

classificar_linhas(df_validado_true['ds_fatos'].iloc[0], ctx_true, ctx_false)

'SIM'